# 모델의 최대 컨텍스트 길이보다 긴 텍스트 임베딩하기

OpenAI의 임베딩 모델은 최대 길이를 초과하는 텍스트를 임베드할 수 없습니다. 최대 길이는 모델에 따라 다르며 문자열 길이가 아닌 _토큰_으로 측정됩니다. 토큰화에 익숙하지 않은 경우 [틱토큰으로 토큰을 계산하는 방법](How_to_count_tokens_with_tiktoken.ipynb)을 참조하세요.

이 노트북에서는 모델의 최대 컨텍스트 길이보다 긴 텍스트를 처리하는 방법을 보여줍니다. 여기서는 `text-embedding-ada-002`의 임베딩을 사용하여 데모하지만, 동일한 아이디어를 다른 모델과 작업에도 적용할 수 있습니다. 임베딩에 대한 자세한 내용은 [OpenAI 임베딩 가이드](https://beta.openai.com/docs/guides/embeddings)에서 확인할 수 있습니다.


## 1. 모델 컨텍스트 길이

먼저 모델을 선택하고 API에서 임베딩을 가져오는 함수를 정의합니다.

In [1]:
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt, retry_if_not_exception_type


EMBEDDING_MODEL = 'text-embedding-ada-002'
EMBEDDING_CTX_LENGTH = 8191
EMBEDDING_ENCODING = 'cl100k_base'

# let's make sure to not retry on an invalid request, because that is what we want to demonstrate
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6), retry=retry_if_not_exception_type(openai.InvalidRequestError))
def get_embedding(text_or_tokens, model=EMBEDDING_MODEL):
    return openai.Embedding.create(input=text_or_tokens, model=model)["data"][0]["embedding"]

텍스트 임베딩-ada-002` 모델의 컨텍스트 길이는 8191 토큰에 `cl100k_base` 인코딩을 사용하며, 이 한도를 초과하면 오류가 발생하는 것을 확인할 수 있습니다.

In [2]:
long_text = 'AGI ' * 5000
try:
    get_embedding(long_text)
except openai.InvalidRequestError as e:
    print(e)

This model's maximum context length is 8191 tokens, however you requested 10001 tokens (10001 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.


특히 많은 임베딩을 프로그래밍 방식으로 처리할 때 이러한 오류를 피하고 싶은 것은 당연합니다. 하지만 여전히 최대 컨텍스트 길이보다 긴 텍스트에 직면할 수 있습니다. 아래에서는 이러한 긴 텍스트를 처리하는 주요 접근 방식에 대한 설명과 레시피를 제공합니다: (1) 텍스트를 허용된 최대 길이로 잘라내는 방법, (2) 텍스트를 청크로 분할하고 각 청크를 개별적으로 임베드하는 방법.

## 1. 입력 텍스트 잘라내기

가장 간단한 해결책은 입력 텍스트를 허용된 최대 길이로 잘라내는 것입니다. 컨텍스트 길이는 토큰 단위로 측정되므로 텍스트를 잘라내기 전에 먼저 토큰화해야 합니다. API는 텍스트 또는 토큰 형태의 입력을 모두 허용하므로 적절한 인코딩을 사용하고 있는지 주의하기만 하면 토큰을 다시 문자열 형태로 변환할 필요가 없습니다. 다음은 이러한 잘림 함수의 예입니다.

In [3]:
import tiktoken

def truncate_text_tokens(text, encoding_name=EMBEDDING_ENCODING, max_tokens=EMBEDDING_CTX_LENGTH):
    """Truncate a string to have `max_tokens` according to the given encoding."""
    encoding = tiktoken.get_encoding(encoding_name)
    return encoding.encode(text)[:max_tokens]

이전의 예제는 이제 오류 없이 작동합니다.

In [4]:
truncated = truncate_text_tokens(long_text)
len(get_embedding(truncated))

1536

## 2. 입력 텍스트 청크 처리

잘라내기가 효과적이지만 관련성이 있을 수 있는 텍스트를 버리는 것은 분명한 단점입니다. 또 다른 접근 방식은 입력 텍스트를 청크로 나눈 다음 각 청크를 개별적으로 임베드하는 것입니다. 그런 다음 청크 임베딩을 개별적으로 사용하거나 평균화(각 청크의 크기에 따라 가중치 부여)와 같은 방식으로 결합할 수 있습니다.

파이썬의 자체 쿡북](https://docs.python.org/3/library/itertools.html#itertools-recipes)에서 시퀀스를 청크로 분할하는 함수를 가져와 보겠습니다.

In [5]:
from itertools import islice

def batched(iterable, n):
    """Batch data into tuples of length n. The last batch may be shorter."""
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := tuple(islice(it, n))):
        yield batch

이제 문자열을 토큰으로 인코딩한 다음 이를 청크로 분할하는 함수를 정의하겠습니다.

In [6]:
def chunked_tokens(text, encoding_name, chunk_length):
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    chunks_iterator = batched(tokens, chunk_length)
    yield from chunks_iterator

마지막으로, 입력 토큰을 청크로 분할하고 각 청크를 개별적으로 임베드하여 입력 텍스트가 최대 컨텍스트 길이보다 긴 경우에도 임베드 요청을 안전하게 처리하는 함수를 작성할 수 있습니다. 평균` 플래그를 `True`로 설정하여 청크 임베딩의 가중 평균을 반환하거나, `False`로 설정하여 수정되지 않은 청크 임베딩 목록을 단순히 반환할 수 있습니다.

In [7]:
import numpy as np


def len_safe_get_embedding(text, model=EMBEDDING_MODEL, max_tokens=EMBEDDING_CTX_LENGTH, encoding_name=EMBEDDING_ENCODING, average=True):
    chunk_embeddings = []
    chunk_lens = []
    for chunk in chunked_tokens(text, encoding_name=encoding_name, chunk_length=max_tokens):
        chunk_embeddings.append(get_embedding(chunk, model=model))
        chunk_lens.append(len(chunk))

    if average:
        chunk_embeddings = np.average(chunk_embeddings, axis=0, weights=chunk_lens)
        chunk_embeddings = chunk_embeddings / np.linalg.norm(chunk_embeddings)  # normalizes length to 1
        chunk_embeddings = chunk_embeddings.tolist()
    return chunk_embeddings

이제 다시 한 번 긴 입력 텍스트를 처리할 수 있게 되었습니다.

In [8]:
average_embedding_vector = len_safe_get_embedding(long_text, average=True)
chunks_embedding_vectors = len_safe_get_embedding(long_text, average=False)

print(f"Setting average=True gives us a single {len(average_embedding_vector)}-dimensional embedding vector for our long text.")
print(f"Setting average=False gives us {len(chunks_embedding_vectors)} embedding vectors, one for each of the chunks.")


Setting average=True gives us a single 1536-dimensional embedding vector for our long text.
Setting average=False gives us 2 embedding vectors, one for each of the chunks.


경우에 따라 텍스트의 의미를 보존하기 위해 단락 경계나 문장 경계에서 청크를 분할하는 것이 좋을 수 있습니다.